<a href="https://colab.research.google.com/github/UMRKawhi/GenChat/blob/master/Week_3_Quiz_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://www.unsw.edu.au/sites/default/files/UNSW_0.png)

# MARK5828 Week 3 Quiz3


---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Linear Regression with VIF on UNICEF Video data (4 marks)

In this question you are required to develop linear regression model with VIF analysis for **both** UNICEF videos published before 2013 and UNICEF videos published after 2013, as given by "**videos_before_2013**" and "**videos_after_2013**". Your job is to use what you learnt in week 3 tutorial about building linear regression model with VIF analysis, and replicate the process using "**videos_before_2013**" and "**videos_after_2013**". Please give your VIF value of the most significant x-variable (smallest p-value) in both cases. 

Please give an valid insight or conclusion about the UNICEF videos published before v.s after year 2013.

In [2]:
# Run this cell to install required software. You may add any other software if you wish
# Upload "week2_helpers.py"
import statsmodels.api as sm
import seaborn as sns; sns.set()
import pandas as pd
# import VIF function from week2_helpers.py
from week2_helpers import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
"""YOU DO NOT NEED TO CHANGE THIS"""


# Read the two CSV files
df_youtube = pd.read_csv('unicef_youtube.csv')
df_indexer = pd.read_csv('unicef_indexer.csv')

# Merge the two CSV files together based on "video IDs" columns
df_merge = df_youtube.merge(df_indexer, left_on = "video_id", right_on = "id")

# Remove unnecessary columns
df_merge = df_merge.drop(columns=["id", "video_id", "url"], axis=1)

# Drop Videos without "likes" information
df_merge = df_merge.dropna(subset=["likes"])

In [4]:
"""YOU DO NO NEED TO CHANGE THIS"""

# Get Videos before & after 2013. Note that .astype(int) converts the year from STRING to INTEGER.
videos_before_2013 = df_merge[df_merge["date_published_year"].astype(int) < 2013]
videos_after_2013 = df_merge[df_merge["date_published_year"].astype(int) >= 2013]


## Q1. Get Dummy Variables for Categorical Columns (0.5 mark)

In [5]:
category_columns = ['genre', 'indexer_source_language', 'date_published_year', 'date_published_month', 'date_published_day']
new_video_before_2013 = pd.get_dummies(videos_before_2013, columns=category_columns, drop_first=True)
new_video_after_2013 = pd.get_dummies(videos_after_2013, columns=category_columns, drop_first=True)

## Q2. Perform Initial Regression (without VIF taken in consideration) (0.5 mark)

In [6]:
# x1 & y1 for before_2013
y1 = new_video_before_2013["likes"]
x1 = new_video_before_2013.drop(['likes'], axis=1).astype(float)
x1 = sm.add_constant(x1)
model1 = sm.OLS(y1, x1)
results1 = model1.fit()
# print(results1.summary())

# x2 & y2 for after_2013
y2 = new_video_after_2013['likes']
x2 = new_video_after_2013.drop(['likes'], axis=1).astype(float)
x2 = sm.add_constant(x2)
model2 = sm.OLS(y2, x2)
results2 = model2.fit()
# print(results2.summary())

## Q3. Get VIF values for each column, and drop columns with their VIF > 5 (0.5 mark) 

In [7]:
# vif1  & df_vif1 for before_2013
vif1 = variance_inflation_factor(x1)
df_vif1 = x1.loc[:, vif1['vif'] <= 5]

# vif2 & df_vif2 for after_2013
vif2 = variance_inflation_factor(x2)
df_vif2 = x2.loc[:, vif2['vif'] <= 5]

## Q4. Perform Regression again (with VIF taken in consideration) (0.5 mark)

In [8]:
# x1 & y1 for before_2013
y1 = new_video_before_2013["likes"]
x1 = sm.add_constant(df_vif1.astype(float))
new_model1 = sm.OLS(y1, x1)
new_results1 = new_model1.fit()
# print(results1.summary())

# x2 & y2 for after 2013
y2 = new_video_after_2013['likes']
x2 = sm.add_constant(df_vif2.astype(float))
new_model2 = sm.OLS(y2, x2)
new_results2 = new_model2.fit()
# print(results2.summary())

# Q5. Please compare the number of significant variables in (1) your initail regression result before any VIF iteration with (2) your final regression result after you finish all VIF iteratio in both cases. Please note that significance level is 0.05. /(1 mark)

Case 1: Before 2013: The top 5 significant variables are: indexer_sentiment_negative, indexer_duration, indexer_ocr_unicef.org, indexer_ocr_unicef@television, indexer_sentiment_neutral.
Their coef are 0.00. After applied the VIF iteration, indexer_ocr_unicef.org,
indexer_ocr_unicef@television	are still at 0.00. However, the p values of indexer_sentiment_negative, indexer_duration, indexer_sentiment_neutral changed to NaN. This means that these variables have been removed.

In [9]:
# Scientific notation forbidden
pd.set_option('display.float_format',lambda x : '%.2f' % x)

# I select the top 5 significant varibales for initial regression
results_as_html = results1.summary().tables[1].as_html()
initial_summary1 = pd.read_html(results_as_html, header=0, index_col=0)[0]
initial_summary1['P>|t|'] = initial_summary1['P>|t|'].astype('float')
initial_summary1 = initial_summary1.sort_values(by=['P>|t|'], ascending=True)
significant_variable = list(initial_summary1.head(6).index)
significant_variable = significant_variable[1:]

# find the top 5 significant variable after using VIF
results_as_html = new_results1.summary().tables[1].as_html()
final_summary1 = pd.read_html(results_as_html, header=0, index_col=0)[0]
final_summary1['P>|t|'] = final_summary1['P>|t|'].astype('float')
final_summary1 = final_summary1.reindex(significant_variable)

In [10]:
# print the result of initial regression
print(significant_variable)
initial_summary1.iloc[1:6]

['indexer_sentiment_negative', 'indexer_duration', 'indexer_ocr_unicef.org', 'indexer_ocr_unicef@television', 'indexer_sentiment_neutral']


,coef,std err,t,P>|t|,[0.025,0.975]
indexer_sentiment_negative,-0.98,0.18,-5.38,0.00,-1.33,-0.62
indexer_duration,0.88,0.10,9.16,0.00,0.69,1.07
indexer_ocr_unicef.org,1.55,0.38,4.13,0.00,0.81,2.29
indexer_ocr_unicef@television,3.36,0.45,7.53,0.00,2.49,4.24
indexer_sentiment_neutral,-0.86,0.09,-9.21,0.00,-1.04,-0.68


In [11]:
# print the result and the p values after VIF
final_summary1

,coef,std err,t,P>|t|,[0.025,0.975]
indexer_sentiment_negative,nan,nan,nan,nan,nan,nan
indexer_duration,nan,nan,nan,nan,nan,nan
indexer_ocr_unicef.org,1.62,0.38,4.27,0.00,0.88,2.37
indexer_ocr_unicef@television,3.56,0.45,7.89,0.00,2.67,4.44
indexer_sentiment_neutral,nan,nan,nan,nan,nan,nan


Case 2: After 2013: The top 5 significant variables are: tags_poverty, indexer_labels_person, indexer_topics_business, tags_celebrity, indexer_topics_social_work.Their coef are 0.00. After applied the VIF iteration, tags_poverty, indexer_topics_business, tags_celebrity, indexer_topics_social_work are still at 0.00. However, the p values of indexer_labels_person changed to NaN. This means that these variables have been removed.

In [12]:
# I select the top 5 significant varibales for initial regression
results_as_html = results2.summary().tables[1].as_html()
initial_summary2 = pd.read_html(results_as_html, header=0, index_col=0)[0]
initial_summary2['P>|t|'] = initial_summary2['P>|t|'].astype('float')
initial_summary2 = initial_summary2.sort_values(by=['P>|t|'], ascending=True)
significant_variable = list(initial_summary2.head(6).index)
significant_variable = significant_variable[1:]

# find the top 5 significant variable after using VIF
results_as_html = new_results2.summary().tables[1].as_html()
final_summary2 = pd.read_html(results_as_html, header=0, index_col=0)[0]
final_summary2['P>|t|'] = final_summary2['P>|t|'].astype('float')
final_summary2 = final_summary2.reindex(significant_variable)

In [13]:
# print the result of initial regression
print(significant_variable)
initial_summary2.iloc[1:6]

['tags_poverty', 'indexer_labels_person', 'indexer_topics_business', 'tags_celebrity', 'indexer_topics_social_work']


,coef,std err,t,P>|t|,[0.025,0.975]
tags_poverty,832.35,217.98,3.82,0.00,404.17,1260.53
indexer_labels_person,5.29,1.66,3.20,0.00,2.04,8.54
indexer_topics_business,5.31,1.55,3.42,0.00,2.26,8.35
tags_celebrity,618.29,208.52,2.96,0.00,208.71,1027.88
indexer_topics_social_work,11.02,3.81,2.89,0.00,3.53,18.51


In [14]:
final_summary2

,coef,std err,t,P>|t|,[0.025,0.975]
tags_poverty,nan,nan,nan,nan,nan,nan
indexer_labels_person,nan,nan,nan,nan,nan,nan
indexer_topics_business,5.32,1.28,4.17,0.00,2.82,7.83
tags_celebrity,nan,nan,nan,nan,nan,nan
indexer_topics_social_work,11.14,2.98,3.74,0.00,5.29,16.99


#Q6. Please give an valid insight or conclusion about the UNICEF videos published before v.s after year 2013. /(1 mark)

By filtering the data, we can observe that the most popular title entitie before 2013 is the football superstar David Backham, and after 2013, the most popular brand is the FC Barcelona. Therefore, we can conclude that when the UNICEF video relates to the topic of football, everyone will be more willing to watch the UNICEF video. However, when the video involves some serious topics, such as family, children, business and refugee issues, the video will lose its appeal.

In [15]:
# x1 & y1 for before_2013
y1 = new_video_before_2013["likes"]
x1 = sm.add_constant(df_vif1.astype(float))
new_model1 = sm.OLS(y1, x1)
new_results1 = new_model1.fit()
# print(results1.summary())

# x2 & y2 for after 2013
y2 = new_video_after_2013['likes']
x2 = sm.add_constant(df_vif2.astype(float))
new_model2 = sm.OLS(y2, x2)
new_results2 = new_model2.fit()
# print(results2.summary())

results_as_html = new_results1.summary().tables[1].as_html()
final_summary1 = pd.read_html(results_as_html, header=0, index_col=0)[0]
final_summary1['P>|t|'] = final_summary1['P>|t|'].astype('float')

results_as_html = new_results2.summary().tables[1].as_html()
final_summary2 = pd.read_html(results_as_html, header=0, index_col=0)[0]
final_summary2['P>|t|'] = final_summary2['P>|t|'].astype('float')

# only consider the result which using VIF iteratio
before_2013 = final_summary1.copy()
after_2013 = final_summary2.copy()
before_2013 = before_2013[before_2013['P>|t|'] <= 0.05]
after_2013 = after_2013[after_2013['P>|t|'] <= 0.05]

# only select two columns: coef and P>|t|
before_2013 = before_2013[['coef', 'P>|t|']]
after_2013 = after_2013[['coef', 'P>|t|']]

In [16]:
# sort values by using coef, print the first 5 rows and last 5 rows
before_2013 = before_2013.sort_values(by='coef', ascending=False)
print(list(before_2013.head().index))
print(list(before_2013.tail().index))

['const', 'title_entities_david_beckham', 'tags_fund', 'description_entities_second', 'tags_poverty']
['tags_childrens', 'description_entities_anja_baron', 'description_entities_thomas_nybo', 'title_entities_haiti', 'is_family_friendly']


In [17]:
# sort values by using coef, print the first 5 rows and last 5 rows
after_2013 = after_2013.sort_values(by='coef', ascending=False)
print(list(after_2013.head().index))
print(list(after_2013.tail().index))

['indexer_brands_fc_barcelona', 'indexer_brands_youtube', 'title_length', 'indexer_num_faces', 'indexer_topics_social_work']
['indexer_topics_social_work', 'indexer_labels_ground', 'indexer_topics_business', 'indexer_topics_dating_and_relationships', 'indexer_topics_non_governmental_organizations_ngos']
